In [5]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [2]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'edpb_guidelines_202003_healthdatascientificresearchcovid19_en.pdf'
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

output_file = "../tmp/covid_health.md"

In [3]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 3, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [8]:
import re
import pandas as pd
file_path = "../../original/covid_health.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)
df_notes = df_notes[df_notes["text"].str.strip() != '']

In [9]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""

section_pattern = re.compile(r'^(\d+) (?!\.)(.+)$')
subsection_pattern = re.compile(r'(\d\.\d)(?!\.)\s*(.*)') 
point_pattern = re.compile(r'^(\d\.\d\.\d)\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^ANNEX (\d+) - (.+)$') # ignore capitalization


annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    if entry.strip() != '':
        section_match = section_pattern.match(entry)
        subsection_match = subsection_pattern.match(entry)
        point_match = point_pattern.match(entry)
        annex_match = annex_pattern.match(entry)

        if section_match:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                match = section_match
                section = match.group(1)
                subsection = ""
                point = ""
                heading = True
                text = match.group(2)
                section_reference = section
                data.append([section, subsection, point, heading, text, section_reference])
        elif subsection_match:
            match = subsection_match
            section = section
            subsection = match.group(1)
            point = ""
            heading = True
            text = match.group(2)
            section_reference = subsection
            data.append([section, subsection, point, heading, text, section_reference])
        elif point_match:
            match = point_match
            section = section
            subsection = subsection
            point = match.group(1)
            heading = True
            text = match.group(2)
            section_reference = point
            data.append([section, subsection, point, heading, text, section_reference])

        elif annex_match:
            match = annex_match
            annex_started = True
            section = "Annex"
            subsection = ""
            point = ""
            heading = True
            text = match.group(1)
            section_reference = section
            data.append([section, subsection, point, heading, text, section_reference])
        
        else:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                section = section
                subsection = subsection
                point = point
                heading = False
                text = entry
                section_reference = section_reference
                
                data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)
df = df.drop([0,1,2,3,4])
df.reset_index(inplace=True)

#df.loc[df["section_reference"] == "",  "section_reference"] = "INTRODUCTION"
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [10]:
df[40:60]
#df.iloc[192]["text"]
#df[df["text"] == " "]

,index,section,subsection,point,heading,text,section_reference
40,45,,,,False,"26. As stipulated in Article 9 (2) (i) GDPR, s...",4.2
41,46,,,,False,"27. Furthermore, such enacted laws must be int...",4.2
42,47,5,,,True,DATA PROTECTION PRINCIPLES,5
43,48,,,,False,28. The principles relating to processing of p...,5
44,49,5,5.1,,True,Transparency and information to data subjects,5.1
45,50,,,,False,29. The principle of transparency means that p...,5.1
46,51,,,,False,"30. In general, a data subject must be individ...",5.1
47,52,,,,False,31. It has to be noted that researchers often ...,5.1
48,53,5,5.1,5.1.1,True,When must the data subject be informed?,5.1.1
49,54,,,,False,32. When personal data have not been obtained ...,5.1.1


In [11]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 5 augmented with footnotes
Row 10 augmented with footnotes
Row 18 augmented with footnotes
Row 27 augmented with footnotes
Row 32 augmented with footnotes
Row 34 augmented with footnotes
Row 35 augmented with footnotes
Row 37 augmented with footnotes
Row 41 augmented with footnotes
Row 55 augmented with footnotes
Row 57 augmented with footnotes
Row 63 augmented with footnotes
Row 68 augmented with footnotes
Row 76 augmented with footnotes
Row 83 augmented with footnotes
Row 86 augmented with footnotes
Row 88 augmented with footnotes
Row 89 augmented with footnotes
Row 90 augmented with footnotes
Row 92 augmented with footnotes
Row 94 augmented with footnotes


In [13]:
file = "../../inputs/documents/covid_health.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [16]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.covid_health
importlib.reload(gdpr_rag.documents.covid_health)
from gdpr_rag.documents.covid_health import CovidHealth

path_to_manual_as_csv_file = "../../inputs/documents/covid_health.parquet"
doc = CovidHealth(path_to_manual_as_csv_file)


In [17]:
from IPython.display import Markdown, display

section = "1"

section = "5.4"
section = "5.1.2"

print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


5 DATA PROTECTION PRINCIPLES
5.1 Transparency and information to data subjects
5.1.2 Exemptions


# 5 DATA PROTECTION PRINCIPLES

## 5.1 Transparency and information to data subjects

### 5.1.2 Exemptions

35. However, Article (14) (5) GDPR stipulates four exemptions of the information obligation. In the current context, the exemption pursuant to Article (14) (5) (b) ("proves impossible or would involve a disproportionate effort") and (c) ("obtaining or disclosure is expressly laid down by Union or Member State law") GDPR are of particular relevance, especially for the information obligation pursuant to Article 14 (4) GDPR.

5.1.2.1 Proves impossible

36. In its Guidelines regarding the principle of Transparency,[^13] the former Article 29-Working-Party has already pointed out that "the situation where it "proves impossible" under Article 14 (5) (b) to provide the information is an all or nothing situation because something is either impossible or it is not; there are no degrees of impossibility. Thus, if a data controller seeks to rely on this exemption it must demonstrate the factors that actually prevent it from providing the information in question to data subjects. If, after a certain period of time, the factors that caused the "impossibility" no longer exist and it becomes possible to provide the information to data subjects then the data controller should immediately do so. In practice, there will be very few situations in which a data controller can demonstrate that it is actually impossible to provide the information to data subjects."

5.1.2.2 Disproportionate effort

37. In determining what constitutes disproportionate effort, Recital 62 refers to the number of data  subjects, the age of the data and appropriate safeguards in place as possible indicative factors. In the Transparency Guidelines mentioned above,[^14] it is recommended that the controller should therefore carry out a balancing exercise to assess the effort involved to provide the information to data subjects against the impact and effects on the data subject if they are not provided with the information.

38. Example: A large number of data subjects where there is no available contact information could be considered as a disproportionate effort to provide the information.

5.1.2.3  Serious impairment of objectives

39. To rely on this exception, data controllers must demonstrate that the provision of the information set out in Article 14 (1) per se would render impossible or seriously impair the achievement of the objectives of the processing.

40. In a case where the exemption of Article (14) (5) (b) GDPR applies, "the controller shall take appropriate measures to protect the data subject's rights and freedoms and legitimate interests, including making the information publicly available".

5.1.2.4 Obtaining or disclosure is expressly laid down by Union or Member State law

41. Article 14 (5) (c) GDPR allows for a derogation of the information requirements in Articles 14 (1), (2) and (4) insofar as the obtaining or disclosure of personal data "is expressly laid down by Union or Member State law to which the controller is subject". This exemption is conditional upon the law in question providing "appropriate measures to protect the data subject's legitimate interests". As stated in the above mentioned Transparency Guidelines,[^15] such law must directly address the data controller and the obtaining or disclosure in question should be mandatory upon the data controller. When relying on this exemption, the EDPB recalls that the data controller must be able to demonstrate how the law in question applies to them and requires them to either obtain or disclose the personal data in question.

  
[^13]: See the Guidelines on transparency under Regulation 2016/679 of the former Article-29 Working-Party from 11.4.2018, WP260 rev.01, 17/EN, page 29 (endorsed by the EDPB). Available at  
[^14]: Guidelines on transparency under Regulation 2016/679 of the former Article-29 Working-Party from 11.4.2018, WP260 rev.01, 17/EN, page 31 (endorsed by the EDPB).  
[^15]: Guidelines on transparency under Regulation 2016/679 of the former Article-29 Working-Party from 11.4.2018, WP260 rev.01, 17/EN, page 32 (endorsed by the EDPB).